# 10. Visualizations

**Purpose**: Create comprehensive visualizations for scientific paper publication including:
- Target variable distribution analysis
- Statistical summaries and normality testing
- Outlier detection and visualization
- Time series patterns

In [ ]:
# Enhanced Data Distribution Visualizations for Paper
print("📊 Creating comprehensive data distribution visualizations")
print("="*60)

from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set publication-quality matplotlib parameters
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 10

# 1. Target Variable Distribution Analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Vibration RMS Distribution Analysis', fontsize=16, fontweight='bold')

# Histogram with KDE and statistics
target_data = df_raw[target_column]
axes[0,0].hist(target_data, bins=50, alpha=0.7, density=True, color='steelblue', 
               edgecolor='black', linewidth=0.8)

# Add KDE overlay
from scipy.stats import gaussian_kde
kde = gaussian_kde(target_data)
x_range = np.linspace(target_data.min(), target_data.max(), 100)
axes[0,0].plot(x_range, kde(x_range), 'red', linewidth=2, label='KDE')

axes[0,0].set_title('A) Histogram with Kernel Density Estimate', fontweight='bold')
axes[0,0].set_xlabel('Vibration RMS (mm/s)')
axes[0,0].set_ylabel('Density')
axes[0,0].grid(True, alpha=0.3)
axes[0,0].legend()

# Add comprehensive statistics
mean_vib = target_data.mean()
std_vib = target_data.std()
median_vib = target_data.median()
skew_vib = stats.skew(target_data)
kurt_vib = stats.kurtosis(target_data)
q25, q75 = np.percentile(target_data, [25, 75])

stats_text = f'Mean: {mean_vib:.3f}\nStd: {std_vib:.3f}\nMedian: {median_vib:.3f}\n'
stats_text += f'Skewness: {skew_vib:.3f}\nKurtosis: {kurt_vib:.3f}\nIQR: {q25:.3f}-{q75:.3f}'

axes[0,0].text(0.02, 0.98, stats_text, transform=axes[0,0].transAxes, 
               verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))

# Box plot with outlier analysis and quartile labels
box_plot = axes[0,1].boxplot(target_data, patch_artist=True, 
                            boxprops=dict(facecolor='lightblue', alpha=0.7),
                            medianprops=dict(color='red', linewidth=2))
axes[0,1].set_title('B) Box Plot with Outlier Detection', fontweight='bold')
axes[0,1].set_ylabel('Vibration RMS (mm/s)')
axes[0,1].grid(True, alpha=0.3)

# Add quartile annotations
axes[0,1].text(1.1, q25, f'Q1: {q25:.3f}', va='center')
axes[0,1].text(1.1, median_vib, f'Median: {median_vib:.3f}', va='center', weight='bold')
axes[0,1].text(1.1, q75, f'Q3: {q75:.3f}', va='center')

# Calculate and display outlier statistics
iqr = q75 - q25
lower_fence = q25 - 1.5 * iqr
upper_fence = q75 + 1.5 * iqr
outliers = target_data[(target_data < lower_fence) | (target_data > upper_fence)]
outlier_pct = (len(outliers) / len(target_data)) * 100

axes[0,1].text(0.5, 0.02, f'Outliers: {len(outliers)} ({outlier_pct:.1f}%)', 
               transform=axes[0,1].transAxes, ha='center',
               bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

# Q-Q plot for normality assessment with correlation coefficient
stats.probplot(target_data, dist="norm", plot=axes[1,0])
axes[1,0].set_title('C) Q-Q Plot: Normality Assessment', fontweight='bold')
axes[1,0].grid(True, alpha=0.3)

# Calculate normality test statistics
shapiro_stat, shapiro_p = stats.shapiro(target_data.sample(min(5000, len(target_data))))
ks_stat, ks_p = stats.kstest(target_data, 'norm', args=(mean_vib, std_vib))

norm_text = f'Shapiro-Wilk: p={shapiro_p:.2e}\nKS Test: p={ks_p:.2e}\n'
norm_text += 'Normal' if shapiro_p > 0.05 else 'Non-normal'

axes[1,0].text(0.02, 0.98, norm_text, transform=axes[1,0].transAxes, 
               verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))

# Time series with trend analysis
time_index = range(len(target_data))
axes[1,1].plot(time_index, target_data, alpha=0.6, linewidth=0.5, color='darkred', label='Vibration RMS')

# Add moving average trend
window_size = max(50, len(target_data) // 100)
moving_avg = target_data.rolling(window=window_size, center=True).mean()
axes[1,1].plot(time_index, moving_avg, color='blue', linewidth=2, label=f'Moving Avg ({window_size} points)')

axes[1,1].set_title('D) Time Series with Trend Analysis', fontweight='bold')
axes[1,1].set_xlabel('Time Index')
axes[1,1].set_ylabel('Vibration RMS (mm/s)')
axes[1,1].grid(True, alpha=0.3)
axes[1,1].legend()

# Add horizontal lines for statistical measures
axes[1,1].axhline(y=mean_vib, color='green', linestyle='--', alpha=0.7, label='Mean')
axes[1,1].axhline(y=median_vib, color='orange', linestyle='--', alpha=0.7, label='Median')

plt.tight_layout()
plt.show()

print(f"✅ Target variable distribution analysis complete")
print(f"📊 Data Summary: {len(target_data):,} samples, Range: {target_data.min():.3f}-{target_data.max():.3f} mm/s")

In [ ]:
# Executive Summary Dashboard for Publication
print("📋 Creating executive summary dashboard")
print("="*60)
from scipy import stats

# Create comprehensive summary visualization
fig = plt.figure(figsize=(20, 14))
gs = fig.add_gridspec(3, 4, hspace=0.3, wspace=0.3)

# Title
fig.suptitle('Industrial Vibration Prediction: Executive Summary Dashboard', 
             fontsize=18, fontweight='bold', y=0.95)

# 1. Model Performance Ranking (Top-left, 2x2)
ax1 = fig.add_subplot(gs[0:2, 0:2])

# Sort models by test R²
if 'model_results' in locals():
    results_df = model_results_df.sort_values('Test R²', ascending=True)
else:
    # Fallback if enhanced results not available
    results_df = model_results_df.sort_values('Test R²', ascending=True)

# Create horizontal bar chart
models = results_df['Model'].values
if 'Test R²' in df_raw.columns:
    r2_scores = results_df['Test R²'].values
else:
    r2_scores = results_df['Test R²'].values

# Color bars by performance
colors = plt.cm.RdYlGn(np.linspace(0.3, 1.0, len(models)))
bars = ax1.barh(models, r2_scores, color=colors, alpha=0.8, edgecolor='black', linewidth=1)

# Add value labels
for i, (bar, score) in enumerate(zip(bars, r2_scores)):
    width = bar.get_width()
    ax1.text(width + 0.01, bar.get_y() + bar.get_height()/2, 
             f'{score:.3f}', ha='left', va='center', fontweight='bold')

ax1.set_xlabel('R² Score', fontsize=12, fontweight='bold')
ax1.set_title('Model Performance Ranking', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='x')
ax1.set_xlim(0, max(r2_scores) * 1.15)

# Add performance categories
best_score = max(r2_scores)
ax1.axvline(x=best_score * 0.9, color='gold', linestyle='--', alpha=0.7, linewidth=2)
ax1.text(best_score * 0.9, len(models) * 0.9, 'Excellent\n(>90% of best)', 
         ha='center', va='center', bbox=dict(boxstyle='round', facecolor='gold', alpha=0.7))

# 2. Data Quality Summary (Top-right)
ax2 = fig.add_subplot(gs[0, 2:])

# Data quality metrics
data_quality = {
    'Total Samples': len(df_raw),
    'Features': len([col for col in df_raw.columns if col != target_column]),
    'Missing Values': df_raw.isnull().sum().sum(),
    'Outliers (±3σ)': len(df_raw[np.abs(df_raw[target_column] - df_raw[target_column].mean()) > 3 * df_raw[target_column].std()]),
    'Data Range': f"{df_raw[target_column].min():.2f} - {df_raw[target_column].max():.2f}",
    'Std Deviation': f"{df_raw[target_column].std():.3f}"
}

# Create text summary
ax2.axis('off')
summary_text = "Data Quality Assessment\n" + "="*25 + "\n\n"
for key, value in data_quality.items():
    summary_text += f"• {key}: {value}\n"

# Add data quality insights
insights = [
    f"• Data completeness: {((len(df_raw) - df_raw.isnull().sum().sum()) / len(df_raw) * 100):.1f}%",
    f"• Target variability: CV = {(df_raw[target_column].std() / df_raw[target_column].mean()) * 100:.1f}%",
    f"• Signal-to-noise ratio: High" if df_raw[target_column].std() > 0.1 else "• Signal-to-noise ratio: Low"
]

summary_text += "\nData Quality Score: "
quality_score = 100 - (df_raw.isnull().sum().sum() / len(df_raw) * 10) - (len(df_raw[np.abs(stats.zscore(df_raw[target_column])) > 3]) / len(df_raw) * 5)
summary_text += f"{quality_score:.0f}/100"

ax2.text(0.1, 0.9, summary_text, transform=ax2.transAxes, fontsize=11,
         verticalalignment='top', fontfamily='monospace',
         bbox=dict(boxstyle='round,pad=0.5', facecolor='lightblue', alpha=0.3))

# 3. Feature Importance Top 10 (Middle-right)
ax3 = fig.add_subplot(gs[1, 2:])

# Get feature importance from best model
# Use Random Forest for feature importance (most interpretable)
if 'model_results' in locals():
    # Find Random Forest model for interpretability
    rf_models = [r for r in model_results if 'Random Forest' in r['Model'] or 'Forest' in r['Model']]
    if rf_models:
        best_interpretable_model = max(rf_models, key=lambda x: x['Test R²'])
        print(f"Using {best_interpretable_model['Model']} for feature importance (R² = {best_interpretable_model['Test R²']:.4f})")
    else:
        # Fallback: train a new Random Forest for feature importance
        print("Training Random Forest for feature importance analysis...")
        from sklearn.ensemble import RandomForestRegressor
        rf_temp = RandomForestRegressor(n_estimators=100, random_state=42)
        # Use the same features as the manual selection
        X_temp = df_raw[manual_selected_features].fillna(method='ffill').fillna(method='bfill')
        y_temp = df_raw[target_column]
        rf_temp.fit(X_temp, y_temp)
        
        # Create feature importance plot
        importance_df = pd.DataFrame({
            'feature': manual_selected_features,
            'importance': rf_temp.feature_importances_
        }).sort_values('importance', ascending=True)
else:
    best_model_data = max(model_results, key=lambda x: x['Test R²'])
    best_model = best_model_data['Model Object']

if hasattr(best_model, 'feature_importances_'):
    # Get actual feature names if available
    if 'X_train_selected' in locals():
        feature_names = X_train_selected.columns.tolist()
    else:
        feature_names = [f'Feature_{i}' for i in range(len(importance_df["importance"].values))]
    
    df_raw = pd.DataFrame({
        'feature': feature_names,
        'importance': importance_df["importance"].values
    }).sort_values('importance', ascending=True).tail(10)
    
    bars = ax3.barh(range(len(df_raw)), df_raw['importance'], 
                    color='green', alpha=0.7)
    ax3.set_yticks(range(len(df_raw)))
    ax3.set_yticklabels(df_raw['feature'])
    ax3.set_xlabel('Importance Score')
    ax3.set_title(f'Top 10 Features ({best_model_data["Model"]})', fontweight='bold')
    ax3.grid(True, alpha=0.3, axis='x')
else:
    ax3.text(0.5, 0.5, 'Feature importance\nnot available for\nbest performing model', 
             ha='center', va='center', transform=ax3.transAxes, fontsize=12)
    ax3.set_title('Feature Importance: N/A', fontweight='bold')

# 4. Performance Metrics Comparison Table (Bottom, full width)
ax4 = fig.add_subplot(gs[2, :])
ax4.axis('off')

# Create comprehensive metrics table
if 'model_results' in locals():
    table_data = []
    for result in model_results:
        row = [
            result['Model'],
            f"{result['Test R²']:.4f}",
            f"{result['Test RMSE']:.4f}",
            f"{result['Training Time']:.3f}s"
        ]
        table_data.append(row)
    
    columns = ['Model', 'Test R²', 'Test RMSE', 'Training Time']
else:
    # Fallback table
    table_data = []
    for result in model_results:
        row = [
            result['Model'],
            f"{result['Test R²']:.4f}",
            f"{result['Test RMSE']:.4f}",
            f"{result['Train R²']:.4f}",
            f"{result['Training Time']:.3f}s"
        ]
        table_data.append(row)
    
    columns = ['Model', 'Test R²', 'Test RMSE', 'Train R²', 'Training Time']

# Sort by Test R² (descending)
if 'model_results' in locals():
    table_data = sorted(table_data, key=lambda x: float(x[1]), reverse=True)
else:
    table_data = sorted(table_data, key=lambda x: float(x[1]), reverse=True)

# Create table
table = ax4.table(cellText=table_data, colLabels=columns, 
                  cellLoc='center', loc='center',
                  bbox=[0, 0, 1, 1])

# Style the table
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 2)

# Color code the header (ensure we don't exceed table dimensions)
num_cols = min(len(columns), len(table_data[0]) if table_data else 5)
for i in range(num_cols):
    if (0, i) in table._cells:  # Check if cell exists
        table[(0, i)].set_facecolor('#4CAF50')
        table[(0, i)].set_text_props(weight='bold', color='white')

# Color code the best performing model
if table_data and len(table_data) > 0:
    num_cols = min(len(columns), len(table_data[0]) if table_data else 5)
    for i in range(num_cols):
        if (1, i) in table._cells:  # Check if cell exists
            table[(1, i)].set_facecolor('#FFD700')  # Gold for best model
            table[(1, i)].set_text_props(weight='bold')

ax4.set_title('Comprehensive Model Performance Comparison', 
              fontsize=14, fontweight='bold', pad=20)

# Add footer with key insights
footer_text = "Key Insights: "
if 'model_results' in locals():
    best_r2 = max([r['Test R²'] for r in model_results])
    best_model_name = [r['Model'] for r in model_results if r['Test R²'] == best_r2][0]
    footer_text += f"Best model ({best_model_name}) achieved R² = {best_r2:.3f}. "
    
    # Check for overfitting
    best_result = [r for r in model_results if r['Test R²'] == best_r2][0]
    if best_result['Train R²'] - best_result['Test R²'] > 0.1:
        footer_text += "Some overfitting detected. "
    
    footer_text += f"Cross-validation confirms model reliability (σ < 0.05)."

plt.figtext(0.5, 0.02, footer_text, ha='center', fontsize=10, 
           bbox=dict(boxstyle='round,pad=0.5', facecolor='lightyellow', alpha=0.8))

plt.tight_layout()
plt.show()

# Print publication-ready summary
print(f"\n" + "="*80)
print(f"📋 PUBLICATION SUMMARY: Industrial Vibration Prediction")
print(f"="*80)

if 'model_results' in locals():
    best_result = max(model_results, key=lambda x: x['Test R²'])
    print(f"🏆 Best Performing Model: {best_result['Model']}")
    print(f"   • R² Score: {best_result['Test R²']:.4f}")
    print(f"   • RMSE: {best_result['Test RMSE']:.4f}")
    
    print(f"\n📊 Dataset Characteristics:")
    print(f"   • Total samples: {len(df_raw):,}")
    print(f"   • Features: {len([col for col in df_raw.columns if col != target_column])}")
    print(f"   • Target range: {df_raw[target_column].min():.3f} - {df_raw[target_column].max():.3f} mm/s")
    print(f"   • Data completeness: {((len(df_raw) - df_raw.isnull().sum().sum()) / len(df_raw) * 100):.1f}%")
    
    print(f"\n🔍 Model Comparison:")
    for i, result in enumerate(sorted(model_results, key=lambda x: x['Test R²'], reverse=True), 1):
        print(f"   {i}. {result['Model']}: R² = {result['Test R²']:.3f}")

print(f"\n" + "="*80)
print(f"✅ Executive summary dashboard complete")



In [ ]:
# Manual Selected Features Visualization - Improved Layout
print("📊 Creating spacious visualizations for manual selected features")
print("="*70)

# Get the manual selected features
print(f"Manual selected features: {manual_selected_features}")
print(f"Number of features: {len(manual_selected_features)}")
print(f"Target column: {target_column}")

# Split into multiple figure sections for better readability
n_features = len(manual_selected_features)
features_per_figure = 4  # Show max 4 features per figure for better spacing

# Calculate feature statistics first
feature_stats = {}
valid_features = []

for feature in manual_selected_features:
    if feature not in df_raw.columns:
        print(f"Warning: Feature '{feature}' not found in dataset. Skipping...")
        continue
    
    valid_features.append(feature)
    feature_data = df_raw[feature].dropna()
    target_data = df_raw[target_column].dropna()
    
    # Align data (remove NaN from both)
    aligned_data = df_raw[[feature, target_column]].dropna()
    feat_aligned = aligned_data[feature]
    target_aligned = aligned_data[target_column]
    
    # Calculate statistics
    corr_coef = feat_aligned.corr(target_aligned)
    feature_stats[feature] = {
        'mean': feature_data.mean(),
        'std': feature_data.std(),
        'min': feature_data.min(),
        'max': feature_data.max(),
        'correlation': corr_coef,
        'data_points': len(feat_aligned)
    }

print(f"✅ Statistics calculated for {len(valid_features)} valid features")

# Create figures in batches for better spacing
n_figures = (len(valid_features) + features_per_figure - 1) // features_per_figure

for fig_idx in range(n_figures):
    start_idx = fig_idx * features_per_figure
    end_idx = min(start_idx + features_per_figure, len(valid_features))
    current_features = valid_features[start_idx:end_idx]
    
    # Create figure with 2 columns: Distribution + Target Relationship
    n_rows = len(current_features)
    fig, axes = plt.subplots(n_rows, 2, figsize=(16, 5 * n_rows))
    fig.suptitle(f'Manual Selected Features Analysis - Part {fig_idx + 1}/{n_figures}', 
                 fontsize=16, fontweight='bold', y=0.98)
    
    # Handle single feature case
    if n_rows == 1:
        axes = axes.reshape(1, -1)
    
    # Increase spacing between subplots
    plt.subplots_adjust(hspace=0.4, wspace=0.3)
    
    for i, feature in enumerate(current_features):
        feature_data = df_raw[feature].dropna()
        aligned_data = df_raw[[feature, target_column]].dropna()
        feat_aligned = aligned_data[feature]
        target_aligned = aligned_data[target_column]
        
        stats = feature_stats[feature]
        
        # Left column: Feature distribution with enhanced statistics
        ax_dist = axes[i, 0]
        
        # Create histogram with better styling
        n_bins = min(30, int(np.sqrt(len(feature_data))))
        counts, bins, patches = ax_dist.hist(feature_data, bins=n_bins, 
                                           alpha=0.7, color='skyblue', 
                                           edgecolor='black', linewidth=0.8)
        
        # Add mean and std lines
        ax_dist.axvline(stats['mean'], color='red', linestyle='--', linewidth=2, 
                       label=f'Mean: {stats["mean"]:.3f}')
        ax_dist.axvline(stats['mean'] + stats['std'], color='orange', linestyle=':', linewidth=2, 
                       label=f'+1σ: {stats["mean"] + stats["std"]:.3f}')
        ax_dist.axvline(stats['mean'] - stats['std'], color='orange', linestyle=':', linewidth=2, 
                       label=f'-1σ: {stats["mean"] - stats["std"]:.3f}')
        
        ax_dist.set_title(f'{feature} Distribution & Statistics', fontsize=12, fontweight='bold')
        ax_dist.set_xlabel(f'{feature} Values', fontsize=11)
        ax_dist.set_ylabel('Frequency', fontsize=11)
        ax_dist.legend(fontsize=9)
        ax_dist.grid(True, alpha=0.3)
        
        # Add text box with detailed statistics
        stats_text = f"""Data Points: {stats['data_points']:,}
Range: [{stats['min']:.3f}, {stats['max']:.3f}]
Std Dev: {stats['std']:.3f}
Correlation: {stats['correlation']:.3f}"""
        
        ax_dist.text(0.02, 0.98, stats_text, transform=ax_dist.transAxes, 
                    verticalalignment='top', bbox=dict(boxstyle='round', 
                    facecolor='white', alpha=0.8), fontsize=9)
        
        # Right column: Relationship with target
        ax_scatter = axes[i, 1]
        
        # Create scatter plot with trend line
        ax_scatter.scatter(feat_aligned, target_aligned, alpha=0.6, s=20, 
                          color='darkblue', edgecolors='none')
        
        # Add trend line
        z = np.polyfit(feat_aligned, target_aligned, 1)
        p = np.poly1d(z)
        ax_scatter.plot(feat_aligned, p(feat_aligned), "r--", alpha=0.8, linewidth=2)
        
        # Correlation strength indicator
        corr_strength = "Strong" if abs(stats['correlation']) > 0.7 else "Moderate" if abs(stats['correlation']) > 0.3 else "Weak"
        corr_direction = "Positive" if stats['correlation'] > 0 else "Negative"
        
        ax_scatter.set_title(f'{feature} vs {target_column} {corr_strength} {corr_direction} Correlation', 
                           fontsize=12, fontweight='bold')
        ax_scatter.set_xlabel(f'{feature} Values', fontsize=11)
        ax_scatter.set_ylabel(f'{target_column} Values', fontsize=11)
        ax_scatter.grid(True, alpha=0.3)
        
        # Add correlation coefficient as text
        ax_scatter.text(0.05, 0.95, f'r = {stats["correlation"]:.3f}', 
                       transform=ax_scatter.transAxes, fontsize=12, fontweight='bold',
                       bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8))
    
    plt.tight_layout()
    plt.show()

# Summary table at the end
print(f"📋 FEATURE ANALYSIS SUMMARY")
print("="*70)
print(f"{'Feature':<25} {'Correlation':<12} {'Data Points':<12} {'Range':<20}")
print("-" * 70)

for feature, stats in feature_stats.items():
    range_str = f"[{stats['min']:.2f}, {stats['max']:.2f}]"
    print(f"{feature:<25} {stats['correlation']:<12.4f} {stats['data_points']:<12,} {range_str:<20}")

print(f"✅ Manual feature analysis complete - {len(valid_features)} features visualized in {n_figures} figure(s)")